In [1]:
import sys
import os
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
sys.path.append("../../")
sys.path.append("../")
from utils.evaluation import evaluate
from utils.metrics import Metrics
from models.NNGenerator import AdaptableDenseModel


In [2]:
import neptune
from neptune_pytorch import NeptuneLogger
from neptune.utils import stringify_unsupported
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pickle
from git import Repo

# Get the git root directory
repo = Repo(".", search_parent_directories=True)
git_root = repo.git.rev_parse("--show-toplevel")

# Load data
X_Train_pd = pickle.load(open(f"{git_root}/data/splits/train/X_pandas.pck", "rb"))
y_Train_pd = pickle.load(open(f"{git_root}/data/splits/train/y_pandas.pck", "rb"))

X_Val_pd = pickle.load(open(f"{git_root}/data/splits/val/X_pandas.pck", "rb"))
y_Val_pd = pickle.load(open(f"{git_root}/data/splits/val/y_pandas.pck", "rb"))

In [4]:
X_Train = torch.tensor(X_Train_pd.values, dtype=torch.float32)
y_Train = torch.tensor(y_Train_pd.values, dtype=torch.float32)

X_Val = torch.tensor(X_Val_pd.values, dtype=torch.float32)
y_Val = torch.tensor(y_Val_pd.values, dtype=torch.float32)

train_dataset = torch.utils.data.TensorDataset(X_Train, y_Train)
val_dataset = torch.utils.data.TensorDataset(X_Val, y_Val)

In [5]:
def label_from_logits(y_hat: torch.Tensor, threshold = 0.5) -> torch.Tensor:
    with torch.no_grad():
        y_pred_tensor = (torch.sigmoid(y_hat) > threshold).float()
    return y_pred_tensor


def evaluate_from_dataframe(X: pd.DataFrame):
    X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
    
    #model: a pytorch model, which transforms X -> y in torch.Tensor format
    model.eval()
    model.cpu()
    y_pred_tensor = label_from_logits(model(X_tensor))
    
    return pd.DataFrame(y_pred_tensor.numpy())


In [6]:
#Adopted from https://forge.ibisc.univ-evry.fr/alacan/GANs-for-transcriptomics/-/blob/master/src/models/utils.py?ref_type=heads

from metrics.aats import compute_AAts
#from metrics.precision_recall import get_precision_recall

def calc_aat(real_data, fake_data):
    _, _, aat = compute_AAts(real_data, fake_data)
    return aat

# def calc_precision_recall(real_data, fake_data):
#     precision, recall = get_precision_recall(real_data, fake_data)
#     return precision, recall

In [7]:

#Adopted from https://forge.ibisc.univ-evry.fr/alacan/GANs-for-transcriptomics/-/blob/master/src/models/utils.py?ref_type=heads
def wasserstein_loss(y_true: torch.tensor, y_pred: torch.tensor):
    """
    Returns Wasserstein loss (product of real/fake labels and critic scores on real or fake data)
    ----
    Parameters:
        y_true (torch.tensor): true labels (either real or fake)
        y_pred (torch.tensor): critic scores on real or fake data
    Returns:
        (torch.tensor): mean product of real labels and critic scores
    """
    return torch.mean(y_true * y_pred)


def generator_loss(fake_score: torch.tensor):
    """
    Returns generator loss i.e the negative scores of the critic on fake data.
    ----
    Parameters:
        fake_score (torch.tensor): critic scores on fake data
    Returns:
        (torch.tensor): generator loss"""

    return wasserstein_loss(-torch.ones_like(fake_score), fake_score)


def discriminator_loss(real_score: torch.tensor, fake_score: torch.tensor):
    """
    Compute and return the wasserstein loss of critic scores on real and fake data i.e: wassertstein_loss = mean(-score_real) + mean(score_fake)
    ----
    Parameters:
        real_score (torch.tensor): critic scores on real data
        fake_score (torch.tensor): critic scores on fake data
    Returns:
        (torch.tensor): wasserstein loss
    """
    real_loss = wasserstein_loss(-torch.ones_like(real_score), real_score)
    fake_loss = wasserstein_loss(torch.ones_like(fake_score), fake_score)

    return real_loss, fake_loss




In [8]:
def training(D, G, d_optimizer, g_optimizer, train_loader, val_loader, epochs, latentSpaceSize, lambda_gp, iters_critic, device, neptune_logger=None, run = None, trial = None):
# Code adopted from https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# and https://github.com/Zeleni9/pytorch-wgan/blob/master/models/wgan_gradient_penalty.py
# and https://forge.ibisc.univ-evry.fr/alacan/GANs-for-transcriptomics/-/blob/master/src/models/utils.py?ref_type=heads
    D = D.to(device)
    G = G.to(device)
    aat = 0

    print("Starting Training Loop...")
    # For each epoch
    for epoch in tqdm(range(epochs)):
        errDItem = 0
        errGItem = 0
        Wasserstein_D = 0
        G.train()
        D.train()

        # For each batch in the dataloader
        for X,y in train_loader:

            y = y.to(device).long()
            
            ############################
            # (1) Update D network: maximize D(x|c) - D(G(z|c)) + lambda_gp * gradient_penalty
            ###########################
            for p in D.parameters():
                p.requires_grad = True  

            for p in G.parameters():
                p.requires_grad = False  # to avoid computation

            # Train with all-real batch
            b_size = X.size(0)
            real_data = X.to(device)

            # Train D
            for crit_step in range(iters_critic):
                d_optimizer.zero_grad()
                
                # Generate batch of latent vectors
                z = torch.randn(b_size, latentSpaceSize, device=device)
                # Generate fake image batch with G
                fake_data = G(z, y)
                NB_GENES = fake_data.size(1)

                # Perform random augmentations for stability
                augmentations = torch.distributions.binomial.Binomial(total_count=1, probs=0).sample(torch.tensor([b_size])).to(device)
                fake_data = fake_data + augmentations[:, None] * torch.normal(0, 0.5, size=(NB_GENES,), device=device)
                real_data = real_data + augmentations[:,None] * torch.normal(0, 0.5, size=(b_size,NB_GENES), device=device)

                # Classify batches with D
                errD_real = D(real_data, y)
                errD_fake = D(fake_data, y)
                # Calculate D's loss on the all-fake batch
                real_loss, fake_loss = discriminator_loss(errD_real, errD_fake)

                # Loss for D in
                d_loss = real_loss + fake_loss
                Wasserstein_D = d_loss.item()

                ####### Gradient penalty #######
                BATCH_SIZE = real_data.size(0)

                # Sample alpha from uniform distribution
                alpha = torch.rand(
                    BATCH_SIZE,
                    1,
                    requires_grad=True,
                    device=real_data.device)

                # Interpolation between real data and fake data.
                interpolation = torch.mul(alpha, real_data) + \
                    torch.mul((1 - alpha), fake_data)

                # Get outputs from critic
                disc_outputs = D(interpolation, y)
                grad_outputs = torch.ones_like(
                    disc_outputs,
                    requires_grad=False,
                    device=real_data.device)

                # Retrieve gradients
                gradients = torch.autograd.grad(
                    outputs=disc_outputs,
                    inputs=interpolation,
                    grad_outputs=grad_outputs,
                    create_graph=True,
                    retain_graph=True)[0]

                # Compute gradient penalty
                gradients = gradients.view(BATCH_SIZE, -1)
                grad_norm = gradients.norm(2, dim=1)

                gradient_penalty = torch.mean((grad_norm - 1) ** 2) * lambda_gp
                d_loss += gradient_penalty
                # Calculate gradients for D in backward pass
                d_loss.backward()

                # Compute error of D as sum of the errors on the real and fake batches and the gradient penalty
                errDItem = d_loss.item()
                # Update D
                d_optimizer.step()

            ############################
            # (2) Update G network: maximize D(G(z|c))
            ###########################
            for p in D.parameters():
                p.requires_grad = False  # to avoid computation

            for p in G.parameters():
                p.requires_grad = True

            g_optimizer.zero_grad()
            
            # Generate batch of latent vectors
            z = torch.randn(b_size, latentSpaceSize, device=device)
            # Generate fake image batch with G
            fake_data = G(z, y)

            NB_GENES = fake_data.shape[1]

            # Perform random augmentations for stability
            augmentations = torch.distributions.binomial.Binomial(total_count=1, probs=0).sample(torch.tensor([BATCH_SIZE])).to(device)
            fake_data = fake_data + augmentations[:, None] * torch.normal(0, 0.5, size=(NB_GENES,), device=device)

            # Calculate G's loss based on this output
            errG = D(fake_data, y)
            
            # Calculate gradients for G
            g_loss =  generator_loss(errG)
            g_loss.backward()
            errGItem += g_loss.item()
            # Update G
            g_optimizer.step()

        # Evaluate the model on the validation set
        if epoch % 20 == 0:
            with torch.no_grad():
                G.eval()
                real_datas = np.array([])
                fake_datas = np.array([])
                for X_val, y_val in val_loader:
                    y_val = y_val.to(device).long()
                    X_val = X_val.to(device).to(torch.float16)
                    z = torch.randn(X_val.size(0), latentSpaceSize)
                    fake_data = G(z.to(device), y_val).to(torch.float16)
                    real_datas = torch.vstack((real_datas, X_val)) if real_datas.size else real_data
                    fake_datas = torch.vstack((fake_datas, fake_data)) if fake_datas.size else fake_data
                aat = calc_aat(real_datas.cpu(), fake_datas.cpu())
            #with torch.device(device):
            #    recall, precision = calc_precision_recall(real_datas, fake_datas)

        errDItem /= len(train_loader)
        errGItem /= len(train_loader)
        Wasserstein_D /= len(train_loader)
        # Log the metrics to neptune
        if neptune_logger is not None:
            run[neptune_logger.base_namespace]['D_Loss'].append(errDItem)
            run[neptune_logger.base_namespace]['G_Loss'].append(errGItem)
            run[neptune_logger.base_namespace]['Wasserstein_D'].append(Wasserstein_D)
            run[neptune_logger.base_namespace]['AAT'].append(aat)
            #run[neptune_logger.base_namespace]['Precision'].append(precision)
            #run[neptune_logger.base_namespace]['Recall'].append(recall)

        print(f"Epoch {epoch} D Loss: {errDItem} G Loss: {errGItem} Wasserstein D: {Wasserstein_D}")
            

In [9]:
class Discriminator(nn.Module):
    def __init__(self, input_size, output_size, class_size, embedding_size):
        super().__init__()
        self.model = nn.Sequential(
            #nn.Linear(input_size, 1024),
            nn.Linear(input_size + embedding_size * class_size, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, output_size),
        )
        self.embedding = nn.Embedding(class_size, embedding_size)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)

    def forward(self, x, cls):
        cls = self.embedding(cls)
        x = torch.cat((x, cls.flatten(start_dim=1)), 1)
        return self.model(x)
    
class Generator(nn.Module):
    def __init__(self, input_size, output_size, class_size, embedding_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size + embedding_size * class_size, 512),
            #nn.Linear(input_size , 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.1),
            nn.Linear(2048, output_size)
        )
        self.embedding = nn.Embedding(class_size, embedding_size)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)

    def forward(self, x, cls):
        cls = self.embedding(cls)
        x = torch.cat((x, cls.flatten(start_dim=1)), 1)
        return self.model(x)

In [10]:
input_dim_data = X_Train.shape[1]
output_dim_data = y_Train.shape[1]
latentSpaceSize = 128

# Define the generator
G = Generator(latentSpaceSize, input_dim_data, output_dim_data, 2)

# Define the discriminator
D = Discriminator(input_dim_data, 16, output_dim_data, 2)





In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
params = {
    "epochs": 800,
    "lambda_gp": 10,
    "latentSpaceSize": latentSpaceSize,
    "device": device,
    "batch_size": 512,
    "lr_g": 0.0001/20,
    "lr_d": 0.001/20,
    "critic_iter": 5,
    "b1": 0.5,
    "b2": 0.999
}




train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1024, shuffle=True)

d_optimizer = torch.optim.Adam(D.parameters(), lr=params["lr_d"], betas=(params["b1"], params["b2"]))
g_optimizer = torch.optim.Adam(G.parameters(), lr=params["lr_g"], betas=(params["b1"], params["b2"]))



In [12]:
run = neptune.init_run(
    api_token=os.getenv("NEPTUNE_API_KEY"),
    project=os.getenv("NEPTUNE_PROJECT_NAME"),
    name="WGAN-GP - 800 - simple",
)

neptune_logger = NeptuneLogger(run=run, model=G)
                               
run[neptune_logger.base_namespace]["hyperparams"] = stringify_unsupported(params)
run[neptune_logger.base_namespace]["G_Structure"] = str(G)
run[neptune_logger.base_namespace]["D_Structure"] = str(D)



training(D, G, d_optimizer, g_optimizer, train_loader , val_loader, params["epochs"], latentSpaceSize, params["lambda_gp"], params["critic_iter"], device, neptune_logger=neptune_logger, run=run)

neptune_logger.log_model()
run.stop()

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/JPL/rna-sequencing/e/RNAS-196
Starting Training Loop...


  0%|          | 0/800 [00:00<?, ?it/s]

c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\torch\autograd\graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 4/4 [00:17<00:00,  4.38s/it]
c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 0 D Loss: 0.0021393661732440226 G Loss: 0.14092089402508903 Wasserstein D: -2.142912024384612e-05
Epoch 1 D Loss: 0.0012957395373524485 G Loss: 0.09353192739344977 Wasserstein D: -3.511208545911562e-05
Epoch 2 D Loss: 0.001123168251731179 G Loss: 0.07730406595693602 Wasserstein D: -0.00013854753199990812
Epoch 3 D Loss: 0.0006770808379966896 G Loss: 0.06878841376596398 Wasserstein D: -0.00023808991992390238
Epoch 4 D Loss: -0.0002166120843453841 G Loss: 0.06163631457757283 Wasserstein D: -0.000785356113960693
Epoch 5 D Loss: -0.0007302447945087939 G Loss: 0.059820024882788425 Wasserstein D: -0.0014415035297820618
Epoch 6 D Loss: -0.0011937784981894326 G Loss: 0.11527941250926131 Wasserstein D: -0.00196809368533688
Epoch 7 D Loss: -0.002564546528396073 G Loss: 0.2163058628777524 Wasserstein D: -0.003004715992854192
Epoch 8 D Loss: -0.003123686030194476 G Loss: 0.36325119737978584 Wasserstein D: -0.0037797239276912663
Epoch 9 D Loss: -0.002816650417301205 G Loss: 0.4767960730966154

...

100%|██████████| 4/4 [00:18<00:00,  4.57s/it]
c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 380 D Loss: -0.3990769553017783 G Loss: -94.26345617120916 Wasserstein D: -0.4215572864025623
Epoch 381 D Loss: -0.4337920609054032 G Loss: -91.47387818023041 Wasserstein D: -0.45675813901674495
Epoch 382 D Loss: -0.4983804342630026 G Loss: -90.18259995467179 Wasserstein D: -0.5129473492815778
Epoch 383 D Loss: -0.3042022598373306 G Loss: -89.82871374383673 Wasserstein D: -0.5139758236758358
Epoch 384 D Loss: -0.5337821586982353 G Loss: -90.24954362349077 Wasserstein D: -0.5458722414670291
Epoch 385 D Loss: 2.791188913625437 G Loss: -127.75915180553089 Wasserstein D: -0.16920001690204328
Epoch 386 D Loss: 0.47329978676109047 G Loss: -130.19839376169486 Wasserstein D: -0.2003907957277098
Epoch 387 D Loss: -0.14673726542012674 G Loss: -129.13623409671382 Wasserstein D: -0.2683507745916193
Epoch 388 D Loss: -0.26614523934317635 G Loss: -123.85174411160129 Wasserstein D: -0.3360420707222465
Epoch 389 D Loss: -0.3650242198597301 G Loss: -119.15073432122077 Wasserstein D: -0.4320815293

100%|██████████| 4/4 [00:18<00:00,  4.56s/it]
c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Julius\Documents\LuH\rna_sequencing\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 400 D Loss: -0.21808871022471182 G Loss: -137.60755018754438 Wasserstein D: -0.3529713237202251
Epoch 401 D Loss: -0.5124682979983883 G Loss: -128.6953130868765 Wasserstein D: -0.542417352849787
Epoch 402 D Loss: -0.5168659770405376 G Loss: -114.25045429576527 Wasserstein D: -0.557783913779092
Epoch 403 D Loss: -0.5751018390788899 G Loss: -112.94977009379781 Wasserstein D: -0.583795240708998
Epoch 404 D Loss: -0.5765340311543925 G Loss: -114.03782611126667 Wasserstein D: -0.5913745906803157
Epoch 405 D Loss: -0.3286699415086866 G Loss: -127.01954864288544 Wasserstein D: -0.47909791319520323
Epoch 406 D Loss: -0.5600785475510818 G Loss: -119.43831565163352 Wasserstein D: -0.5810081641990822
Epoch 407 D Loss: 0.14971971845293378 G Loss: -115.13709056294047 Wasserstein D: -0.39410197651469625
Epoch 408 D Loss: -0.509437507682747 G Loss: -120.6946178516308 Wasserstein D: -0.5563265927188046
Epoch 409 D Loss: -0.4120310003107244 G Loss: -117.30743472226017 Wasserstein D: -0.5990324753

KeyboardInterrupt: 

In [ ]:
# Save the model
#torch.save(G.state_dict(), f"{git_root}/experiments/generating/models/G_800_WGAN_GP.pt")
#torch.save(D.state_dict(), f"{git_root}/experiments/generating/models/D_800_WGAN_GP.pt")

In [13]:
G.eval()

Generator(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Linear(in_features=512, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=1024, out_features=2048, bias=True)
    (7): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.1)
    (9): Linear(in_features=2048, out_features=5045, bias=True)
  )
  (embedding): Embedding(105, 2)
)

In [14]:
G(torch.randn(1, latentSpaceSize, device=device), y_Val[0].long().unsqueeze(dim=0).to(device))

tensor([[-0.1270,  0.0696, -0.2601,  ..., -1.1326, -0.1175,  0.0132]],
       device='cuda:0')

In [15]:
X_Val[0]

tensor([0.0000, 0.0000, 0.0000,  ..., 3.3789, 0.0000, 0.0000])

In [16]:
# Classify the discriminator on the first 50 validation data points
with torch.no_grad():
    D.eval().cpu()
    y_pred = D(X_Val[:50], y_Val[:50].long())
    print(y_pred.mean())



tensor(22.0881)


In [17]:
# Generate 50 samples
with torch.no_grad():
    G.eval().cpu()
    fake_data = G(torch.randn(50, latentSpaceSize), y_Val[:50].long())
    print(fake_data)

# Evaluate the generated data on the discriminator
with torch.no_grad():
    D.eval().cpu()
    y_pred = D(fake_data, y_Val[:50].long())
    print(y_pred.mean())

tensor([[ 2.1170e-01, -7.4901e-01,  2.0443e-01,  ...,  2.1142e+00,
          5.0957e-01,  7.0062e-01],
        [ 8.7831e-03,  1.4698e-01, -9.8590e-02,  ..., -1.6326e+00,
         -1.4369e-02, -3.0847e-01],
        [ 3.7625e-01, -1.4402e-01, -2.8685e-01,  ...,  2.5300e+00,
         -4.7173e-01, -3.8767e-01],
        ...,
        [-9.4485e-02,  3.4533e-01,  3.4342e-01,  ..., -1.6533e+00,
          1.4203e-01, -2.1923e-01],
        [ 4.0288e-01,  9.9806e-02,  2.4194e-01,  ..., -2.2537e-01,
         -3.7465e-02,  1.4169e-03],
        [ 8.6841e-01, -2.4582e-01,  4.3270e-02,  ...,  1.8616e+00,
          1.0579e-01, -5.7536e-02]])
tensor(240.7790)


The discriminator classifies the generated output clearly as fake. Fake is positiv